#Intro

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
!pip3 install gensim


cpu
    100% |████████████████████████████████| 23.6MB 1.2MB/s 
    100% |████████████████████████████████| 1.4MB 12.0MB/s 
    100% |████████████████████████████████| 133kB 29.4MB/s 
    100% |████████████████████████████████| 61kB 22.4MB/s 
    100% |████████████████████████████████| 4.7MB 6.9MB/s 
    100% |████████████████████████████████| 552kB 26.0MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import itertools
from gensim import models
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from scipy.sparse import csr_matrix

import random



import pickle

import os
sep = os.sep
import os.path

import pandas as pd

import time


if(os.path.isfile('utils.py')):
    !rm 'utils.py'
    print("deleted")

exchangedrive = drive.CreateFile({'id':'12djgLg96tD7ynEhImcqoYnL5ZamenDEM'})
exchangedrive.GetContentFile('utils.py')

from utils import *


#Download Data

In [0]:
exchangedrive = drive.CreateFile({'id':'1cMCzlvTMlUPgaYaUIdlMVtTpp8r10GnX'})
exchangedrive.GetContentFile('robosclean.p')


exchangedrive = drive.CreateFile({'id':'1A5Wa6LiaGs8XeW2S2qjGbcoSSlsMGW97'})
exchangedrive.GetContentFile('tweetsLAtrain.csv')
exchangedrive = drive.CreateFile({'id':'1CrUCS7oWzdvYoFwtWgikiGkDAu6w3dOF'})
exchangedrive.GetContentFile('tweetsLAtest.csv')
exchangedrive = drive.CreateFile({'id':'1AU_saqdyRxtgMX2TC7Q42OBIUm68cdw_'})
exchangedrive.GetContentFile('tweetsLA.csv')



exchangedrive = drive.CreateFile({'id':'1NuSVM7-h0CCRtzi0woM4h5tVd6T7JO2C'})
exchangedrive.GetContentFile('tweetsNYtrain.csv')
exchangedrive = drive.CreateFile({'id':'1UYZY0sh1-Q8MIofMAHDGaNKv8cfDA0ui'})
exchangedrive.GetContentFile('tweetsNYtest.csv')


exchangedrive = drive.CreateFile({'id':'1Jv19eJTZwsZWEA_rUudvKwcn1TS-DFBa'})
exchangedrive.GetContentFile('tweets2016_2half.csv')
exchangedrive = drive.CreateFile({'id':'1E8WlhOXb3tfQbLUpizx7QXzeydv57eXN'})
exchangedrive.GetContentFile('toy_2017_Jan.csv')



#Models

In [0]:
class RepreAsDoc():
    def __init__(self):
        pass


    def fit(self, tuples_train, tf_idf = False, lda = False):#t is a tuple (date_out, coor_out, tweets_out)

        ldates_as_doc = []# a list of [wordidx,...,wordidx]), where the index of the list is a dateidx or a cooridx or a wordidx
        lcoor_as_doc = []# a list of [wordidx,...,wordidx]), where the index of the list is a cooridx

        self.max_idxdate = max([tuple[0] for tuple in tuples_train])
        #print(self.max_idxdate)
        self.min_idxword = max([tuple[1] for tuple in tuples_train]) + 1
        #print(self.min_idxword)


        max_idxword = 0
        for tuple in tuples_train:
            for idxword in tuple[2]:
                if idxword > max_idxword:
                    max_idxword = idxword
        #print(max_idxword)
        self.max_idxword = max_idxword


        min_idxword = max_idxword
        for tuple in tuples_train:
            for idxword in tuple[2]:
                if idxword < min_idxword:
                    min_idxword = idxword
        #print(min_idxword)

        vectors = [[] for _ in range(max_idxword+1)]

        for i in range(max_idxword+1):
            vectors[i] = [0 for _ in range(max_idxword-min_idxword + 1)]

        for tuple in tuples_train:
            dateidx, cooridx, wordsidxs = tuple[0], tuple[1], tuple[2]
            for idxword in wordsidxs:
                vectors[dateidx][idxword-min_idxword] += 1
                vectors[cooridx][idxword-min_idxword] += 1

                for word_idx in wordsidxs:
                    vectors[word_idx][idxword-min_idxword] += 1

        if(tf_idf and not lda):
            tfidf_model = TfidfTransformer()
            vectors = tfidf_model.fit_transform(vectors)
            vectors = vectors.todense()

        if(tf_idf and lda):
            tfidf_model = TfidfTransformer()
            vectors = tfidf_model.fit_transform(vectors)

            lda_model = LatentDirichletAllocation(n_components=20, max_iter=5,
                                            learning_method='online',
                                            learning_offset=50.,
                                            random_state=0)
            vectors = lda_model.fit_transform(vectors)

        if (not tf_idf and lda):

            vectors = csr_matrix(vectors)

            lda_model = LatentDirichletAllocation(n_components=20, max_iter=5,
                                                  learning_method='online',
                                                  learning_offset=50.,
                                                  random_state=0)
            vectors = lda_model.fit_transform(vectors)

        self.W = np.array(vectors)

    def predict(self, t_idx, l_idx, w_idx):
        vec_t = self.W[t_idx].reshape(1, -1)
        vec_l = self.W[l_idx].reshape(1, -1)

        vec_w = np.average([self.W[idx].reshape(1, -1) for idx in w_idx], axis=0)

        vectors = [vec_t, vec_l, vec_w]
        #print(vec_t.shape, vec_l.shape, vec_w.shape)
        #for vec1, vec2 in itertools.combinations(vectors, r=2):
        #    if (cosine_similarity(vec1, vec2).shape != (1, 1)):
        #        print('cosine_similarity shape error')
        #    print(cosine_similarity(vec1, vec2))
        score = sum([cosine_similarity(vec1, vec2)[0][0] for vec1, vec2 in itertools.combinations(vectors, r=2)])
        return round(score, 6)

    def getPlaces(self):
        return self.W[self.max_idxdate+1:self.min_idxword,:]

    def queryItem(self, idxs, number= 10):
        query_vectors = self.W[idxs]
        query_vectors.reshape(query_vectors.shape[0],1,-1)
        #print(self.max_idxdate+1)

        date_vectors = self.W[list(range(0,self.max_idxdate+1))]
        #print(date_vectors)

        date_vectors.reshape(date_vectors.shape[0], 1, -1)

        cosines_sim_date = cosine_similarity(query_vectors, date_vectors)



        #coor_vectors = self.W[list(range(self.max_idxdate+1,self.min_idxword))]
        coor_vectors = self.W[self.max_idxdate+1:self.min_idxword,:]
        #print(coor_vectors)

        coor_vectors.reshape(coor_vectors.shape[0], 1, -1)
        cosines_sim_coor = cosine_similarity(query_vectors, coor_vectors)

        word_vectors = self.W[list(range(self.min_idxword, self.max_idxword + 1))]
        word_vectors.reshape(word_vectors.shape[0], 1, -1)
        cosines_sim_word = cosine_similarity(query_vectors, word_vectors)


        indexes_date = np.argsort(cosines_sim_date, )
        indexes_coor = np.argsort(cosines_sim_coor) + self.max_idxdate+1
        indexes_word = np.argsort(cosines_sim_word) + self.min_idxword

        return (indexes_date[:1,-number:], indexes_coor[:1,-number:], indexes_word[:1,-number:])

    def queryVector(self, query_vectors, number= 10):
        query_vectors = np.array(query_vectors)
        query_vectors.reshape(query_vectors.shape[0],1,-1)
        #print(self.max_idxdate+1)

        date_vectors = self.W[list(range(0,self.max_idxdate+1))]
        #print(date_vectors)

        date_vectors.reshape(date_vectors.shape[0], 1, -1)

        cosines_sim_date = cosine_similarity(query_vectors, date_vectors)



        #coor_vectors = self.W[list(range(self.max_idxdate+1,self.min_idxword))]
        coor_vectors = self.W[self.max_idxdate+1:self.min_idxword,:]
        #print(coor_vectors)

        coor_vectors.reshape(coor_vectors.shape[0], 1, -1)
        cosines_sim_coor = cosine_similarity(query_vectors, coor_vectors)

        word_vectors = self.W[list(range(self.min_idxword, self.max_idxword + 1))]
        word_vectors.reshape(word_vectors.shape[0], 1, -1)
        cosines_sim_word = cosine_similarity(query_vectors, word_vectors)


        indexes_date = np.argsort(cosines_sim_date, )
        indexes_coor = np.argsort(cosines_sim_coor) + self.max_idxdate+1
        indexes_word = np.argsort(cosines_sim_word) + self.min_idxword

        return (indexes_date[:1,-number:], indexes_coor[:1,-number:], indexes_word[:1,-number:])


#Eval

In [0]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}

namesets = ['Santiago','LA']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[]}



for i in range(1):
    trainST,word2idxST,testST = chargeData_SplitTrainTest('tweets2016_2half.csv')
    trainLA,word2idxLA,testLA = chargeData_SplitTrainTest("tweetsLA.csv")
    
    trainsets = [trainST,  trainLA]
    testsets = [testST,  testLA]
    for train,test,name in list(zip(trainsets, testsets, namesets)):
        QE = QuantitativeEvaluator()


        model = RepreAsDoc()

        ldadict={True:'lda', False:''}
        tfidfdict={True:'tf_idf', False:''}

        for lda in [False, True]:
            for tfidf in [False, True]:
                print(lda,tfidf)
                model.fit(train, lda=lda, tf_idf=tfidf)

                print('Done Training')


                for predict_type in 'wlt':
                    QE.get_ranks(test, model, predict_type = predict_type)
                    mrr1 = QE.compute_mrr()[0]
                    print(f"{name} MRR {predict_type} {mrr1}")
                    results[translate[predict_type]].append(mrr1)
                results['Dataset'].append(name)
                results['Model'].append(str(type(model))+'_'+tfidfdict[tfidf]+'_'+ldadict[lda])
                df = pd.DataFrame(results)

                df.to_csv('resultsRobos.df')
    
    
    
    

False False
Done Training
Santiago MRR w 0.6049
Santiago MRR l 0.446
Santiago MRR t 0.3959
False True
Done Training
Santiago MRR w 0.449
Santiago MRR l 0.4356
Santiago MRR t 0.4147
True False
Done Training
Santiago MRR w 0.4261
Santiago MRR l 0.4276
Santiago MRR t 0.4128
True True
Done Training
Santiago MRR w 0.4042
Santiago MRR l 0.3933
Santiago MRR t 0.3828
False False
Done Training
LA MRR w 0.4468
LA MRR l 0.4316
LA MRR t 0.3904
False True
Done Training
LA MRR w 0.4123
LA MRR l 0.42
LA MRR t 0.4021
True False
Done Training
LA MRR w 0.4076
LA MRR l 0.4077
LA MRR t 0.3967
True True
Done Training
LA MRR w 0.3847
LA MRR l 0.3752
LA MRR t 0.3662
CPU times: user 5h 42min 53s, sys: 4min 34s, total: 5h 47min 27s
Wall time: 5h 41min 57s


#Consulta a los datos de AACH

In [7]:
%%time
indexerRob = Indexer()
trainRob = indexerRob.fit_transform(data+'robosclean.p', dates_vocab_mincount=0, words_vocab_mincount=10, places_vocab_mincount=0)

CPU times: user 1.46 s, sys: 59.8 ms, total: 1.52 s
Wall time: 1.54 s


In [11]:
print(len(trainRob[1]))

4006


In [12]:
%%time
AD_rob = RepreAsDoc()
AD_rob.fit(trainRob[0])

CPU times: user 6.81 s, sys: 266 ms, total: 7.08 s
Wall time: 7.08 s


In [0]:
index = indexerRob.Item2index('portonazo')

In [0]:
(hours, timesp, words) = AD_rob.queryItem([index], number=10)

hours = list(reversed(hours[0]))
timesp = list(reversed(timesp[0]))
words = list(reversed(words[0]))

timesp = [(indexerRob.idx2item[index][1]/100,indexerRob.idx2item[index][0]/100) for index in timesp]
words = [indexerRob.idx2item[index] for index in words]

In [26]:
print(list(hours))
[print(lat,',',lon) for lat,lon in timesp]
print(words)

[0, 23, 22, 21, 1, 3, 20, 4, 2, 19]
-33.56 , -70.56
-33.51 , -70.66
-33.47 , -70.55
-33.53 , -70.65
-33.58 , -70.59
-33.59 , -70.58
-33.58 , -70.56
-33.55 , -70.59
-33.46 , -70.72
-33.53 , -70.66
['portonazo', 'entrando', 'victima', 'robo', 'intimidacion', 'llegando', 'ingresar', 'domicilio', 'porton', 'ingresando']


#Consulta a satos de Twitter usando los datos de AACH como referencia.

In [0]:
df = pd.read_pickle(data+'robosclean.p')

In [0]:
df['coor'] = list(zip(list(df['latitude']),list(df['longitude'])))

In [29]:
counter1 = Counter(df['coor'])
pairs1 = counter1.most_common(2)
pairs1

[((-70.8852447, -33.6041295), 168), ((-71.3187696, -31.7613364), 106)]

In [0]:
dftest = df[df['coor']==(-70.8852447, -33.6041295)]

In [0]:
df = pd.read_csv(data+'tweets2016_2half.csv')

In [0]:
df.reset_index(inplace=True)

In [35]:
%%time
indexerST = Indexer()
trainST = indexerST.fit_transform(data+'tweets2016_2half.csv', dates_vocab_mincount=0, words_vocab_mincount=100, places_vocab_mincount=50)


CPU times: user 10.3 s, sys: 346 ms, total: 10.7 s
Wall time: 10.7 s


In [36]:
%%time
AD_ST = RepreAsDoc()
AD_ST.fit(trainST[0])

CPU times: user 3.58 s, sys: 11.2 ms, total: 3.59 s
Wall time: 3.59 s


In [0]:
def QueryItem(item, model, indexer, most_close=10):
    index = indexer.Item2index(item)
    (hours, timesp, words) = model.queryItem([index], number=most_close)
    hours = list(reversed(hours[0]))
    timesp = list(reversed(timesp[0]))
    words = list(reversed(words[0]))
    timesp = [(indexer.idx2item[index][0]/100,indexer.idx2item[index][1]/100) for index in timesp]
    words = [indexer.idx2item[index] for index in words]
    print(list(hours))
    [print(lat,',',lon) for lat,lon in timesp]
    print(words)

In [41]:
QueryItem((-7088,-3360), AD_ST, indexerST, most_close=20)

[3, 14, 2, 15, 1, 13, 4, 0, 16, 23, 5, 20, 21, 18, 17, 22, 19, 12, 6, 10]
-33.44 , -70.62
-31.0 , -71.0
-33.42 , -70.59
-33.45 , -70.62
-33.4 , -70.58
-33.43 , -70.63
-29.95 , -71.26
-29.91 , -71.26
-33.49 , -70.75
-33.61 , -70.88
-38.76 , -72.75
-33.4 , -70.59
-33.44 , -70.6
-37.46 , -73.34
-33.43 , -70.57
-33.44 , -70.58
-29.93 , -71.25
-38.42 , -72.37
-33.42 , -70.62
-33.47 , -70.72
['nimo', '_http_', 'entrenasano', 'floramour', 'semana', 'miraxhobbies', 'jugueteria', 'mircoles', 'chile', 'hoy', 'buen', 'partiendo', 'santiago', 'buenosdias', 'mejor', 'energa', 'comenzando', 'comenzamos', 'comenzar', 'snapchat']
